# // RANDOM FOREST CLASSIFIER //

#Importing necessary libraries for data preprocessing

In [1]:
import pandas as pd #For reading the dataset and to perform framing functions if needed
import numpy as np  #linear algebra function
import matplotlib.pyplot as plt #for visualizing the data
import seaborn as sns #for data distribution

#Importing the dataset
I have used a 'Bank dataset of credit card' to perform this Random forest classiication

In [2]:
#---> Replace the path name for your path name
bank_data=pd.read_csv('C:/Users/ADMIN/Documents/Data science algorithms/bank-full.csv')

In [3]:
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Target
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
bank_data.shape

(45211, 17)

In [5]:
summary={'Columns':bank_data.columns,
        'Unique Values':[bank_data[col].nunique() for col in bank_data.columns],
        'Details':[bank_data[col].unique() for col in bank_data.columns]}

In [6]:
details=pd.DataFrame(summary)
details

,Columns,Unique Values,Details
0,age,77,"[58, 44, 33, 47, 35, 28, 42, 43, 41, 29, 53, 5..."
1,job,12,"[management, technician, entrepreneur, blue-co..."
2,marital,3,"[married, single, divorced]"
3,education,4,"[tertiary, secondary, unknown, primary]"
4,default,2,"[no, yes]"
5,balance,7168,"[2143, 29, 2, 1506, 1, 231, 447, 121, 593, 270..."
6,housing,2,"[yes, no]"
7,loan,2,"[no, yes]"
8,contact,3,"[unknown, cellular, telephone]"
9,day,31,"[5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 19, 20, 21..."


In [7]:
bank_data.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
Target       0
dtype: int64

In [8]:
new_data=bank_data.drop(['marital','contact','day','month','campaign'],axis=1)
new_data.head()

,age,job,education,default,balance,housing,loan,duration,pdays,previous,poutcome,Target
0,58,management,tertiary,no,2143,yes,no,261,-1,0,unknown,no
1,44,technician,secondary,no,29,yes,no,151,-1,0,unknown,no
2,33,entrepreneur,secondary,no,2,yes,yes,76,-1,0,unknown,no
3,47,blue-collar,unknown,no,1506,yes,no,92,-1,0,unknown,no
4,33,unknown,unknown,no,1,no,no,198,-1,0,unknown,no


#To perform the decision tree classifier, we need to encode strings into numerical values

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encode=LabelEncoder()
new_data['job_encode']=label_encode.fit_transform(new_data['job'])
new_data['education_encode']=label_encode.fit_transform(new_data['education'])
new_data['default_encode']=label_encode.fit_transform(new_data['default'])
new_data['housing_encode']=label_encode.fit_transform(new_data['housing'])
new_data['loan_encode']=label_encode.fit_transform(new_data['loan'])
new_data['poutcome_encode']=label_encode.fit_transform(new_data['poutcome'])

In [10]:
x=new_data[['age','job_encode','education_encode','default_encode','balance','housing_encode','loan_encode','duration','pdays','previous','poutcome_encode']]
x.head()           

,age,job_encode,education_encode,default_encode,balance,housing_encode,loan_encode,duration,pdays,previous,poutcome_encode
0,58,4,2,0,2143,1,0,261,-1,0,3
1,44,9,1,0,29,1,0,151,-1,0,3
2,33,2,1,0,2,1,1,76,-1,0,3
3,47,1,3,0,1506,1,0,92,-1,0,3
4,33,11,3,0,1,0,0,198,-1,0,3


In [11]:
y=new_data.Target.replace({'no':0,'yes':1})
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: int64

#splitting the datset into training and testing 

In [12]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=100)

#Training the Random forest classifier model

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
classifier=RandomForestClassifier()

In [14]:
parameters={'n_estimators':[100,150,200,250],'criterion':['gini','entropy']}

In [15]:
forest_classifier=GridSearchCV(classifier,parameters,scoring='accuracy',cv=5)
forest_classifier.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [100, 150, 200, 250]},
             scoring='accuracy')

In [19]:
forest_classifier.best_params_

{'criterion': 'entropy', 'n_estimators': 250}

In [16]:
classifications=forest_classifier.predict(x_train)
classifications.flatten()

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_train,classifications))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     27937
           1       1.00      1.00      1.00      3710

    accuracy                           1.00     31647
   macro avg       1.00      1.00      1.00     31647
weighted avg       1.00      1.00      1.00     31647



In [18]:
print(confusion_matrix(y_train,classifications))

[[27937     0]
 [    0  3710]]


This model secured an accuracy of 100% for classifying the datas from the above bank data